In [4]:
import os
import fitz
import re
import json 
from ollama import Client
import faiss
import pandas as pd
import numpy as np
import Stemmer
from tqdm import tqdm
import gradio as gr

from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Document
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.retrievers import BaseRetriever, QueryFusionRetriever
from llama_index.core.schema import TextNode, NodeWithScore
from llama_index.retrievers.bm25 import BM25Retriever

# Connect to Ollama Server

In [2]:
client = Client(
  host='http://localhost:11434',
)

# Ingestion

In [3]:
# Path to the dataset folder
DATASET_PATH = r"C:\Users\LEGION\Desktop\Project\AI351\RAG LTO\lto_rag_reviewer\notebooks\extracted_text.json"

def get_text_and_metadata(input_path):
    """Load text and metadata from a file and perform chunking."""
    with open(input_path, "r", encoding="utf-8") as f:
        extracted_data = json.load(f)

    texts = []
    metadata = []

    for entry in extracted_data:
        text = entry["text"]
        source_metadata = {
            "source": entry["source"],
            "folder": entry["folder"],
            "file_name": entry["file_name"],
            "page": entry["page"],
            "title": entry["title"],
            "url": entry["url"]
        }
        texts.append(text)
        metadata.append(source_metadata)

    return texts, metadata

In [110]:
docs, metadatas = get_text_and_metadata(DATASET_PATH)

In [102]:
documents = [Document(text=docs[t], metadata=metadatas[t]) for t in range(len(docs))]
splitter = TokenTextSplitter(
    chunk_size=512,
    chunk_overlap=20,
    separator=" ",
)
nodes = splitter.get_nodes_from_documents(documents)

# Embedding and Retrieval

## Dense via FAISS

In [103]:
def generate_embeddings(nodes, client, model):
    embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
    # Generate embeddings for documents using Ollama
    for doc in tqdm(nodes):
        response = embed_model.get_text_embedding(doc.text)
        doc.embedding = response
    return nodes

In [104]:
class FaissIndexer:
    """
    Faiss-based indexer for efficient similarity search using inner-product (cosine) similarity.

    This class handles the creation and management of a FAISS index from node embeddings.
    
    :ivar faiss_index: The FAISS index for storing and querying embeddings.
    :vartype faiss_index: faiss.IndexFlatIP
    :ivar embedding_dim: Dimensionality of the embeddings.
    :vartype embedding_dim: int
    """

    def __init__(self):
        """
        Initialize the FaissIndexer class.

        :ivar faiss_index: The FAISS index, initialized as None.
        :ivar embedding_dim: The dimension of embeddings, initialized as None.
        """
        self.faiss_index = None
        self.embedding_dim = None

    def normalize_embeddings(self, embeddings):
        """
        Normalize embeddings to have unit L2 norm.

        :param embeddings: Array of embeddings to normalize.
        :type embeddings: np.ndarray
        :return: Normalized embeddings.
        :rtype: np.ndarray
        """
        return embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

    def build_index(self, nodes):
        """
        Build the FAISS index from a list of nodes containing embeddings.

        :param nodes: List of nodes, where each node contains an `embedding` attribute.
        :type nodes: list
        :raises ValueError: If the nodes list is empty or embeddings are inconsistent.
        """
        if not nodes:
            raise ValueError("Nodes list cannot be empty.")
        
        embeddings = np.array([np.array(node.embedding) for node in nodes])
        normalized_embeddings = self.normalize_embeddings(embeddings)

        self.embedding_dim = normalized_embeddings[0].shape[0]
        self.faiss_index = faiss.IndexFlatIP(self.embedding_dim)  # Inner-product similarity
        self.faiss_index.add(normalized_embeddings)

    def get_index(self):
        """
        Get the FAISS index instance.

        :return: The FAISS index used for similarity search.
        :rtype: faiss.IndexFlatIP
        :raises ValueError: If the index has not been built.
        """
        if self.faiss_index is None:
            raise ValueError("Index has not been built yet. Call 'build_index' first.")
        return self.faiss_index

In [105]:
class FAISSVectorStoreRetriever(BaseRetriever):
    def __init__(self, faiss_index, documents):
        """
        Initialize the FAISS retriever.
        :param faiss_index: The FAISS index containing precomputed embeddings.
        :param documents: List of document chunks.
        :param embeddings: Precomputed embeddings corresponding to the document chunks.
        """
        self.faiss_index = faiss_index
        self.documents = documents

    def _retrieve(self, query_embedding, top_k=5):
        """
        Retrieve the top-k nearest neighbors using the FAISS index.
        :param query_embedding: The embedding of the query.
        :param top_k: Number of top results to retrieve.
        """

        norm_query_embedding = np.array([query_embedding])
        norm_query_embedding /= np.linalg.norm(norm_query_embedding, axis=1, keepdims=True)

        distances, indices = self.faiss_index.search(norm_query_embedding, top_k)
        retrieved_docs = [
            NodeWithScore(node=self.documents[idx], score=1 - dist)
            for idx, dist in zip(indices[0], distances[0])
            if idx != -1
        ]
        return retrieved_docs

In [107]:
# def generate_embeddings(nodes, client, model):
#     # Generate embeddings for documents using Ollama
#     for doc in tqdm(nodes):
#         response = client.embeddings(prompt=doc.text, model=model)
#         doc.embedding = response["embedding"]
#     return nodes

In [108]:
# embedding
nodes_embed = generate_embeddings(nodes, client, "mxbai-embed-large")

 21%|██        | 2525/12302 [02:16<08:47, 18.53it/s]


KeyboardInterrupt: 

In [106]:
# # embedding

# nodes_embed = generate_embeddings(nodes, client, "BAAI/bge-small-en-v1.5")

  0%|          | 24/12302 [00:05<44:36,  4.59it/s]


KeyboardInterrupt: 

In [15]:
#indexing
index = FaissIndexer()
index.build_index(nodes_embed)
faiss_index = index.get_index()

faiss_retriever = FAISSVectorStoreRetriever(faiss_index=faiss_index,documents=nodes_embed)

## Sparse Embedding via BM25

In [11]:
# bm25_retriever = BM25Retriever.from_defaults(
#     nodes=nodes,
#     similarity_top_k=5,
#     stemmer=Stemmer.Stemmer("english"),
#     language="english",
# )

## Hybrid Retrieval via Reciprocal Rank

In [16]:
def hybrid_embedding(results: dict, top_k: int):
    x = QueryFusionRetriever
    ranked_results = QueryFusionRetriever._reciprocal_rerank_fusion(x, results)
    return ranked_results[:top_k]

# Post Retrieval

## Summarization

In [17]:
def summarize_each_chunk(nodes, client, query, model="llama3.2", parent=False):
    if parent:
        chunks = [doc.text for doc in nodes]
    else:
        chunks = [doc.node.text for doc in nodes]
    summaries = []
    
    for i, chunk in enumerate(chunks):
        prompt = f"""
        Summarize the following text in one concise paragraph, focusing on key points relevant to the query: "{query}".
        
        - Emphasize information directly related to the query.
        - Exclude unrelated, redundant, or speculative details.
        - Do NOT introduce new information or answer the query itself. 
        
        Text:
        {chunk}
        
        Summary:
        """
        
        response = client.generate(model=model, prompt=prompt)
        summary = response['response'].strip()
        summaries.append(summary)

    return summaries

# Generation

In [20]:
def generate_response_with_notice(summaries, query, client, model="llama3.2"):
    # Combine summaries into context block
    context = "\n".join(summaries)
    
    # Create prompt to answer based on summarized text
    prompt = f"""
    Use the following summarized information to answer the query accurately and concisely. 
    DO NOT USE BACKGROUND KNOWLEDGE OUTSIDE THE CONTEXT PROVIDED.
    If the information is not sufficient to fully address the query, respond ONLY with:
    "The available information is insufficient to provide a complete answer to this query."

    Summarized Context:
    {context}
    
    Query:
    {query}
    
    Response:
    """
    
    # Send the prompt to Ollama
    response = client.generate(
        model=model,
        prompt=prompt
    )
    
    return response['response'].strip()

In [26]:
# def generate_response_with_notice(processed_query, query, client, model="llama3.2"):
#     # Combine summaries into context block
#     context = "\n".join(summaries)
    
#     # Create prompt to answer based on summarized text
#     prompt = f"""
#     Use the following summarized information to answer the query accurately and concisely. 
#     DO NOT USE BACKGROUND KNOWLEDGE OUTSIDE THE CONTEXT PROVIDED.
#     If the information is not sufficient to fully address the query, respond ONLY with:
#     "The available information is insufficient to provide a complete answer to this query."

#     Summarized Context:
#     {context}
    
#     Query:
#     {query}
    
#     Response:
#     """
    
#     # Send the prompt to Ollama
#     response = client.generate(
#         model=model,
#         prompt=prompt
#     )
    
#     return response['response'].strip()

In [37]:
def generate_response_with_notice(processed_query, client, model="llama3.2"):
    # Combine summaries into context block
    # context = "\n".join(summaries)
    
    # Create prompt to answer based on summarized text
    prompt = f"""
    Instruction:
    {processed_query}
    
    Response:
    """
    
    # Send the prompt to Ollama
    response = client.generate(
        model=model,
        prompt=prompt
    )
    
    return response['response'].strip()

# Querying

## Query Transforms

In [72]:
# Few-shot examples
FEW_SHOTS = """


Example 2:
Question: What will happen when your front tire blows out?
A. The back end will sway towards the side of the blowout
B. The back end will sway away from the blowout
C. The front end will pull towards the side of the blowout
D. The front end will pull to the opposite side of the blowout
Please answer only in letters
Correct Answer: [C]

Example 3:
Question: What should you do when an ambulance comes up behind you flashing red lights and/or sounding its siren?
A. Stop as soon as you can
B. Maintain your speed, let the ambulance driver will find a way around you
C. Speed up so that you don't hold the ambulance
D. Pull over to the right and slow down or even stop if necessary
Please answer only in letters
Correct Answer: [D]
"""

def paraphrase_query(query,client,model):

    few_shot_examples = '''
    Example 1:
    Transform this into a Retrieval Augmented Generation query: What is AI?
    Output: "Retrieve information on artificial intelligence and provide a concise definition."

    Example 2:
    Transform this into a Retrieval Augmented Generation query: How does reinforcement learning work?
    Output: "Retrieve explanations of reinforcement learning algorithms with examples."

    Example 3:
    Transform this into a Retrieval Augmented Generation query: Explain neural networks.
    Output: "Retrieve details on neural networks and explain their structure and function."

    Now, transform the following query and provide the response inside double quotes:
    '''

    full_prompt = few_shot_examples + f"\nQuery: {query}"

    response = client.generate(
        model=model,
        prompt=full_prompt
    )
    # Extract text within double quotes
    matches = re.findall(r'"([^"]*)"', response['response'])

    print(f"Query: {query}")
    print(f"Transformed: {matches if matches else 'No match found'}")
    print('-' * 60)

    if matches:
        return matches[0]
    else:
        return query


def cot_qa_format(query, context, client, model):
    return f"""
    Let's break down the problem step by step:
    1. Identify the key components of the following question: {paraphrase_query(query, client, model)}
    2. Analyze relevant facts and eliminate unnecessary details.
    3. Provide the best possible answer based on the reasoning process.

    Here is relevant context about the query:
    {context}
    Based on this information, answer the following question:
    {query}
    """


def answer_all_prompts(prompts_with_choices, prompts, mode="hybrid"):
    results = []
    answers = []

    i=0

    # Initialize retrievers (dense and sparse)
    dense_retriever = dense_index.as_retriever(similarity_top_k=top_k)
    sparse_retriever = BM25Retriever.from_defaults(docstore=docstore, similarity_top_k=top_k)
    
    # Wrap the prompt list with tqdm to track progress
    for prompt, prompt_with_choice in tqdm(zip(prompts, prompts_with_choices), desc="Processing Prompts", total=len(prompts)):
        try:
            # 1. Retrieve relevant documents based on mode
            if mode == "dense":
                retrieved_docs = dense_retriever.retrieve(prompt)
            elif mode == "sparse":
                retrieved_docs = sparse_retriever.retrieve(prompt)
            else:  # Hybrid mode (default using reciprocal rerank fusion)
                fusion_retriever = QueryFusionRetriever(
                    retrievers=[dense_retriever, sparse_retriever], 
                    similarity_top_k=top_k, 
                    num_queries=1, 
                    mode="reciprocal_rerank", 
                    use_async=True, 
                    verbose=True, 
                    llm=None
                )
                ranked_results = fusion_retriever.retrieve(prompt)
                retrieved_docs = ranked_results

            # 2. Prepare context from retrieved documents
            combined_context = "\n\n".join([doc.text for doc in retrieved_docs])

            # 3. Apply Chain-of-Thought (CoT) prompt transformation
            new_query = cot_qa_format(prompt, combined_context)
            #print(new_query)
            #print("\n\nPerforming query response...")
            result1 = ollama_llm.complete(new_query)
            result2 = ollama_llm.complete(new_query)
            result3 = ollama_llm.complete(new_query)
            #print(f"result 1: {result1}")
            #print(f"result 2: {result2}")
            #print(f"result 3: {result3}")

            # Aggregate results for final prompt
            final_query_with_results = f"""
            Based on the results:
            - Result 1: {result1.text}
            - Result 2: {result2.text}
            - Result 3: {result3.text}

            Now, synthesize these results and answer this question:
            {paraphrase_query(prompt) + "And answer which of the choices match the synthesized results."}
            {prompt_with_choice}

            \nPlease answer only in letters and put them inside a bracket '[]'. Don't add anything but the letters. No explanation. If you don't see the answer from the choices using the information you retrieved, use background knowledge.
            Here are two examples on how to answer the question:
            {FEW_SHOTS}

            """
            #print(final_query_with_results)
            answer = ollama_llm.complete(final_query_with_results)
            print(answer)

            # 4. Format References
            ref_text = "\n".join(
                [f"Source {i+1}: {doc.metadata['title']} (Page {doc.metadata['page']}, Folder: {doc.metadata['folder']})"
                 for i, doc in enumerate(retrieved_docs)]
            )

            # 5. Store results
            results.append({
                "question": prompt,
                "answer": answer.text,
                "references": ref_text
            })

            answers.append(answer.text)

        except Exception as e:
            results.append({
                "question": prompt,
                "answer": str(e),
                "references": ""
            })
            print(f"Error processing prompt: {e}")

        # i+=1
        # if i==1:
        #     return answers
    
    return answers

## Query Generation

In [22]:
docstore = {}

# Store documents using full metadata as the key
for doc in documents:
    key = tuple(doc.metadata.items())  # Convert metadata to tuple for hashable key
    docstore[key] = doc

In [23]:
def get_document_by_chunk_metadata(chunk_node):
    # Convert chunk metadata to tuple for matching
    metadata_key = tuple(chunk_node.metadata.items())

    # Retrieve document from docstore
    document = docstore.get(metadata_key)
    return document

In [24]:
def remove_duplicate_documents(doc_list):
    seen_ids = set()
    unique_docs = []

    for doc in doc_list:
        if doc.doc_id not in seen_ids:
            seen_ids.add(doc.doc_id)
            unique_docs.append(doc)

    return unique_docs

In [95]:
def gen_query(query, top_k, client, mode='dense', summary=False, model="llama3.2", chunks_only=True):
    embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

    response = embed_model.get_text_embedding(query)

    query_embedding = response

    top_k_docs = faiss_retriever._retrieve(query_embedding, top_k=top_k)

    bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=top_k,
    stemmer=Stemmer.Stemmer("english"),
    language="english",
    )
    retrieved_nodes = bm25_retriever.retrieve(query)

    results = {'faiss': top_k_docs, 'bm25':retrieved_nodes}
    ranked_results = hybrid_embedding(results, top_k=top_k)

    if mode == 'dense':
        print('using FAISS')
        ans_nodes =top_k_docs
    elif mode == 'sparse':
        print('using BM25')
        ans_nodes = retrieved_nodes
    else:
        print('using Hybrid')
        ans_nodes = ranked_results

    parent_flag = True
    context = set([get_document_by_chunk_metadata(docs).text for docs in ans_nodes])
    if chunks_only:
        parent_flag = False
        print('using chunks only')
        context = [docs.node.text for docs in ans_nodes]
        
    if summary:
        print('using summaries')
        context_nodes = remove_duplicate_documents([get_document_by_chunk_metadata(docs) for docs in ans_nodes])

        if chunks_only:
            context_nodes=ans_nodes
        summaries = summarize_each_chunk(context_nodes, client, model='llama3.2:latest', query=query,parent=parent_flag)
        context = summaries
    instruction = cot_qa_format(query, context,client, model='llama3.2:latest')
    # print(f"context: {instruction}")
    # print(f"instruction: {instruction}")
    # answer = generate_response_with_notice(instruction, client, model=model)
    
    answer = generate_response_with_notice(query, client, model=model)
    # print(f"Query: {query}")
    print(f"Answer: {answer}")
    print('-' * 60)
    # Format the references
    references = []
    for i, doc in enumerate(ranked_results[:top_k], start=1):
        metadata = doc.metadata
        source_info = f"Source {i}: {metadata['title']} (Page {metadata['page']}, Folder: {metadata['folder']})"
        references.append(source_info)

    return answer, "\n".join(references)

# Evaluation

In [26]:
# Generate prompts dynamically
def generate_prompt(row):
    options = []
    for choice in ['A', 'B', 'C', 'D', 'E']:
        # Check for NaN or blank values
        if pd.notna(row[choice]) and row[choice] != '':
            options.append(f"{choice}. {row[choice]}")
    
    # Construct the prompt with few-shot examples
    prompt = f"\nActual Question: {row['Question']}\n" + "\n".join(options)
    prompt += "\nPlease answer only in letters and put them inside a bracket '[]'. If the question contains the statement 'Check all that apply' then add comma separator if there are multiple answers ONLY IF ALLOWED."
    
    return prompt

In [27]:
# Load the Excel file
file_path = 'LTO_EXAM.csv'
df = pd.read_csv(file_path)
df['Prompt'] = df.apply(generate_prompt, axis=1)
display(df.head())

,Question,A,B,C,D,E,Answer,Prompt
0,What should you do in case your vehicle breaks...,Open your trunk and hood,Stand on the expressway and flag down passing ...,Call for help using a mobile phone or an expre...,Park as far to the right as possible,Put your hazard warning light on,"A, C, D, E",\nActual Question: What should you do in case ...
1,What will happen when your front tire blows out?,The back end will sway towards the side of the...,The back end will sway away from the blowout,The front end will pull towards the side of th...,The front end will pull to the opposite side o...,NaN,C,\nActual Question: What will happen when your ...
2,What should you do when an ambulance comes up ...,Stop as soon as you can,"Maintain your speed, let the ambulance driver ...",Speed up so that you don't hold the ambulance,Pull over to the right and slow down or even s...,NaN,D,\nActual Question: What should you do when an ...
3,While driving the hood of your car lifts up bl...,Look through the gap underneath the hood or ou...,Brake suddenly so you don't leave the road,Pull to the side of the road and refasten the ...,Turn your headlights on and look out of the si...,NaN,"A,C",\nActual Question: While driving the hood of y...
4,"In case of an accident, the first duty of the ...",pick-up the injured person and take him to the...,report the accident to the nearest hospital,report the accident to the nearest police station,NaN,NaN,A,"\nActual Question: In case of an accident, the..."


In [94]:
i=0
gen_query(df.loc[i,"Prompt"], top_k=15, client=client, mode='dense', model="llama3.1:8b",chunks_only=True )

using FAISS
using chunks only
Query: 
Actual Question: What should you do in case your vehicle breaks down on an expressway? Check all that apply.
A. Open your trunk and hood
B. Stand on the expressway and flag down passing drivers for help
C. Call for help using a mobile phone or an expressway phone
D. Park as far to the right as possible
E. Put your hazard warning light on
Please answer only in letters and put them inside a bracket '[]'. If the question contains the statement 'Check all that apply' then add comma separator if there are multiple answers ONLY IF ALLOWED.
Transformed: ['Retrieve information on road emergencies, specifically what to do when a vehicle breaks down on an expressway.']
------------------------------------------------------------
Query: 
Actual Question: What should you do in case your vehicle breaks down on an expressway? Check all that apply.
A. Open your trunk and hood
B. Stand on the expressway and flag down passing drivers for help
C. Call for help using

('[C, D, E]',
 "Source 1: how-to-drive-safely-expressway (Page 2, Folder: LTO Portal)\nSource 2: Filipino Driver's Manual: Volume 1 (2nd Edition) (Page 134, Folder: Driver’s License)\nSource 3: right-side-driving (Page 3, Folder: LTO Portal)\nSource 4: before-moving-car-parked-position-you-should (Page 1, Folder: LTO Portal)\nSource 5: how-to-make-u-turn (Page 3, Folder: LTO Portal)\nSource 6: cclex-cebu-cordova-link-expressway (Page 4, Folder: LTO Portal)\nSource 7: what-should-driver-light-vehicle-before-overtaking-truck (Page 2, Folder: LTO Portal)\nSource 8: how-to-drive-safely-expressway (Page 1, Folder: LTO Portal)\nSource 9: what-to-do-brakes-fail-driving (Page 2, Folder: LTO Portal)\nSource 10: Filipino Driver's Manual: Volume 1 (2nd Edition) (Page 152, Folder: Driver’s License)\nSource 11: Filipino Driver's Manual: Volume 1 (2nd Edition) (Page 114, Folder: Driver’s License)\nSource 12: what-to-do-emergency-vehicle-following-you-intersection (Page 2, Folder: LTO Portal)\nSource

In [96]:
qr_range = (0,91)
df["AI"] = np.nan
ai_answer = []
for i in tqdm(range(*qr_range)):
    ai_answer.append(gen_query(df.loc[i,"Prompt"], top_k=15, client=client, mode='dense', model="llama3.1:8b",chunks_only=True ))

df.loc[qr_range[0]:qr_range[1]-1, "AI"] = [answ[0] for answ in ai_answer]
df.loc[qr_range[0]:qr_range[1]-1, ["Question","Answer","AI"]]

  0%|          | 0/91 [00:00<?, ?it/s]

using FAISS
using chunks only
Query: 
Actual Question: What should you do in case your vehicle breaks down on an expressway? Check all that apply.
A. Open your trunk and hood
B. Stand on the expressway and flag down passing drivers for help
C. Call for help using a mobile phone or an expressway phone
D. Park as far to the right as possible
E. Put your hazard warning light on
Please answer only in letters and put them inside a bracket '[]'. If the question contains the statement 'Check all that apply' then add comma separator if there are multiple answers ONLY IF ALLOWED.
Transformed: ['Retrieve information on emergency procedures for broken down vehicles, specifically those occurring on expressways, and list appropriate actions.', "['A', 'C', 'E']"]
------------------------------------------------------------


  1%|          | 1/91 [00:32<48:08, 32.09s/it]

Answer: [A, D, E]
------------------------------------------------------------
using FAISS
using chunks only
Query: 
Actual Question: What will happen when your front tire blows out?
A. The back end will sway towards the side of the blowout
B. The back end will sway away from the blowout
C. The front end will pull towards the side of the blowout
D. The front end will pull to the opposite side of the blowout
Please answer only in letters and put them inside a bracket '[]'. If the question contains the statement 'Check all that apply' then add comma separator if there are multiple answers ONLY IF ALLOWED.
Transformed: ["Retrieve information on vehicle safety after a tire blowout and provide possible effects on the vehicle's movement."]
------------------------------------------------------------


  2%|▏         | 2/91 [01:01<45:04, 30.39s/it]

...

Answer: [B]
------------------------------------------------------------
using FAISS
using chunks only
Query: 
Actual Question: If the driver is turning left, he must: (U-turn / Left Rule)
A. have the right of way
B. do so slowly with caution
C. yield to approaching cars
Please answer only in letters and put them inside a bracket '[]'. If the question contains the statement 'Check all that apply' then add comma separator if there are multiple answers ONLY IF ALLOWED.
Transformed: ['Retrieve traffic rules for left turns and explain the relevant laws. In [A], [B], or [C]?']
------------------------------------------------------------


 77%|███████▋  | 70/91 [36:00<11:26, 32.71s/it]

Answer: [B]
------------------------------------------------------------
using FAISS
using chunks only
Query: 
Actual Question: You may not cross a single broken white or yellow line:
A. whent urning left into a driveway
B. when passing to the right on a one-way street
C. when to do so would interfene with traffic
Please answer only in letters and put them inside a bracket '[]'. If the question contains the statement 'Check all that apply' then add comma separator if there are multiple answers ONLY IF ALLOWED.
Transformed: ['Retrieve rules related to traffic signs, specifically broken white or yellow lines, with explanations of when it is allowed to cross them.', 'You may not cross a single broken white or yellow line: [A] when turning left into a driveway, [B] when passing to the right on a one-way street.']
------------------------------------------------------------


 77%|███████▋  | 70/91 [36:32<10:57, 31.32s/it]


RemoteProtocolError: Server disconnected without sending a response.

# Visualization

In [90]:
df[["Question","Answer","AI"]].to_csv('results_v3_with_query_transform_model_318b.csv',index=False)

In [42]:
df['Answer'].iloc[0]

'A, C, D, E'

In [24]:
# Gradio interface with dynamic model, mode selection, and top_k slider
iface = gr.Interface(
    fn=lambda query, top_k, model, mode: gen_query(
        query=query,
        top_k=top_k,
        client=client,
        mode=mode,
        model=model
    ),
    inputs=[
        gr.Textbox(label="Enter your query"),
        gr.Slider(1, 20, value=5, step=1, label="Top K Results"),  # Slider for top_k (1 to 20)
        gr.Dropdown(
            choices=["llama3.3", "llama3.2:latest", "llama3.1:8b"],
            value="llama3.3",
            label="Select Model"
        ),
        gr.Dropdown(
            choices=["hybrid", "dense", "sparse"],
            value="hybrid",
            label="Select Retrieval Mode"
        )
    ],
    outputs=[
        gr.Textbox(label="Answer", lines=6),
        gr.Textbox(label="References", lines=10),
    ],
    title="RAG System with LLaMA Models",
    description=(
        "Ask questions and get answers with references from PDF documents. "
        "Adjust Top-K to control the number of retrieved chunks. "
        "Choose different models and retrieval modes for customization."
    )
)

# Launch the Gradio interface
iface.launch()

* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


In [23]:
# Handle closing behavior
def on_close():
    iface.close()
on_close()

Closing server running on port: 7867
